# Процессы

In [2]:
import random
import os
import time
import threading
import multiprocessing
import os

In [3]:
# Типы задач
def cpu_bound_task(n):
    def fib(n):
        if n == 0:
            return 0
        elif n == 1:
            return 1
        return fib(n-1) + fib(n-2)

    print(f'{n} -> {fib(n)}')


def io_bound_task(ind):
    print(f'start task {ind}')
    time.sleep(5)
    print(f'finish task {ind}\n', end='')

In [6]:
%%time

# CPU bound задача на потоках
tasks = []

for number in [36, 35, 34, 33, 32, 31, 30, 25, 20, 15, 10]:
    task = threading.Thread(target=cpu_bound_task, args=(number,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

31 -> 1346269
20 -> 6765
15 -> 610
10 -> 55
25 -> 75025
30 -> 832040
32 -> 2178309
33 -> 3524578
34 -> 5702887
35 -> 9227465
36 -> 14930352
CPU times: user 28.4 s, sys: 253 ms, total: 28.7 s
Wall time: 28.3 s


In [7]:
%%time

# CPU bound задача на процессах
tasks = []

for number in [36, 35, 34, 33, 32, 31, 30, 25, 20, 15, 10]:
    task = multiprocessing.Process(target=cpu_bound_task, args=(number,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()


15 -> 610
10 -> 55
20 -> 6765
25 -> 75025
30 -> 832040
31 -> 1346269
32 -> 2178309
33 -> 3524578
34 -> 5702887
35 -> 9227465
36 -> 14930352
CPU times: user 32.2 ms, sys: 56.3 ms, total: 88.6 ms
Wall time: 10.3 s


In [8]:
%%time

# IO bound задача на потоках
tasks = []

for ind in range(10):
    task = threading.Thread(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

start task 0
start task 1start task 2

start task 3start task 4
start task 5

start task 6
start task 7
start task 8
start task 9
finish task 0
finish task 1
finish task 5
finish task 7
finish task 3
finish task 6
finish task 2
finish task 4
finish task 9
finish task 8
CPU times: user 12 ms, sys: 7.99 ms, total: 20 ms
Wall time: 5.01 s


In [9]:
%%time

# IO bound задача на процессах
tasks = []

for ind in range(10):
    task = multiprocessing.Process(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

start task 0
start task 1
start task 2
start task 3
start task 4
start task 5
start task 6
start task 7
start task 8
start task 9
finish task 0
finish task 2
finish task 1
finish task 3
finish task 4
finish task 5
finish task 6
finish task 7
finish task 8
finish task 9
CPU times: user 62.8 ms, sys: 56.6 ms, total: 119 ms
Wall time: 5.12 s


In [12]:
# Получение идентификатора процесса
def io_bound_task(ind):
    print(f'start task {ind} with {os.getpid()}')
    time.sleep(60)
    print(f'finish task {ind}')
    
tasks = []

for ind in range(10):
    task = multiprocessing.Process(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

start task 1 with 1479
start task 2 with 1482
start task 0 with 1477
start task 3 with 1495
start task 4 with 1498
start task 5 with 1503
start task 6 with 1509
start task 7 with 1512
start task 8 with 1514
start task 9 with 1517
finish task 0
finish task 1
finish task 2
finish task 3
finish task 4
finish task 5
finish task 6
finish task 8
finish task 7
finish task 9


In [13]:
# Пулл процессов
from multiprocessing import Pool

def getpid(n):
    time.sleep(2)
    return os.getpid()

with Pool(3) as p:
    print(p.map(getpid, range(5)))

[1548, 1549, 1550, 1548, 1550]


In [14]:
# Шаринг ресурсов
share_memory = {
    'count': 0,
}
def share_memory_task():
    print(f'read {share_memory["count"]}')
    share_memory['count'] += 1
    print(f'write {share_memory["count"]}')

tasks = []

for _ in range(5):
    task = multiprocessing.Process(target=share_memory_task)
    tasks.append(task)
    task.start()
    
for task in tasks:
    task.join()

read 0
read 0
read 0
write 1
write 1
write 1
read 0
read 0
write 1
write 1


In [15]:
# Шаринг ресурсов через файл
filename = 'share_memory.tmp'

with open(filename, 'w') as fd:
    fd.write('0')

def share_memory_task():
    with open(filename) as fd:
        count = int(fd.read())
    print(f'read {count}')

    with open(filename, 'w') as fd:
        fd.write(str(count + 1))
    print(f'write {count + 1}')


tasks = []

for _ in range(10):
    task = multiprocessing.Process(target=share_memory_task)
    tasks.append(task)
    task.start()
    
for task in tasks:
    task.join()

with open(filename) as fd:
    print(f'COUNT: {fd.read()}')

read 0
write 1
read 0
write 1
read 0
write 1
read 1
read 1
read 2
read 2
write 2
write 3
write 2
write 3
read 3
read 3
write 4
write 4
read 4
write 5
COUNT: 5


In [19]:
# Шаринг ресурсов через файл с блокировкой
filename = 'share_memory.tmp'

# TODO исправить ошибку
# Ищите помощь в локументации https://docs.python.org/3.6/library/multiprocessing.html
from multiprocessing import Lock

l = Lock()
with open(filename, 'w') as fd:
    fd.write('0')

def share_memory_task(lock):
    with lock:
        with open(filename) as fd:
            count = int(fd.read())
        print(f'read {count}')

        with open(filename, 'w') as fd:
            fd.write(str(count + 1))
        print(f'write {count + 1}')

tasks = []
for _ in range(10):
    task = multiprocessing.Process(target=share_memory_task(l))
    tasks.append(task)
    task.start()
    
for task in tasks:
    task.join()

with open(filename) as fd:
    print(f'COUNT: {fd.read()}')

read 0
write 1
read 1
write 2
read 2
write 3
read 3
write 4
read 4
write 5
read 5
write 6
read 6
write 7
read 7
write 8
read 8
write 9
read 9
write 10
COUNT: 10


In [23]:
# Шаринг ресурсов через очередь
from multiprocessing import Queue

q = Queue()
q.put(0)

def share_memory_task(q):
    count = q.get()
    print(f'read {count}')

    q.put(count + 1)
    print(f'write {count + 1}')


tasks = []

for _ in range(10):
    task = multiprocessing.Process(target=share_memory_task, args=(q,))
    tasks.append(task)
    task.start()
    
for task in tasks:
    task.join()

print(f'COUNT: {q.get()}')

read 0
write 1
read 1
write 2
read 2
read 3
write 3
read 4
write 4
write 5
read 5
write 6
read 6
write 7
read 7
write 8
read 8
write 9
read 9
write 10
COUNT: 10


# Ассинхронность

In [27]:
import asyncio
import random
import time

loop = asyncio.get_event_loop()

async def sleep(delay):
    await asyncio.sleep(delay)

In [28]:
async def io_bound_task(group_name):
    print(f'group is "{group_name}"')
    await sleep(2)
    count = random.randint(100, 200)
    print(f'count {count}')
    return count

In [37]:
%%time

# Последовательное выполнение
async def main():
    count = 0
    for name in ['Amir', 'Zarina', 'Misha', 'Ilya', 'Igor']:
        count += await io_bound_task(name)
    print(f'COUNT: {count}')

loop.run_until_complete(main())

RuntimeError: This event loop is already running

group is "Amir"
count 193
group is "Zarina"
count 186
group is "Misha"
count 132
group is "Ilya"
count 111
group is "Igor"
count 108
COUNT: 730


In [38]:
%%time

# Совместное выполнение
async def main():
    tasks = []
    count = 0
    for name in ['Amir', 'Zarina', 'Misha', 'Ilya', 'Igor']:
        tasks.append(io_bound_task(name))
        
    results = await asyncio.gather(*tasks)
    print(f'COUNT: {sum(results)}')

loop.run_until_complete(main())

RuntimeError: This event loop is already running

group is "Amir"
group is "Zarina"
group is "Misha"
group is "Ilya"
group is "Igor"
count 177
count 149
count 194
count 108
count 104
COUNT: 732


In [39]:
%%time

# Блокирущая операция
async def sleep(delay):
    time.sleep(delay)

loop.run_until_complete(main())

RuntimeError: This event loop is already running

group is "Amir"
count 114
group is "Zarina"
count 152
group is "Misha"
count 100
group is "Ilya"
count 147
group is "Igor"
count 103
COUNT: 616


In [40]:
# Обработка ошибок
async def sleep(delay):
    await asyncio.sleep(delay)
    
async def error_task():
    raise ValueError 
    
async def main():
    tasks = [error_task()]
    for name in ['Amir', 'Zarina', 'Misha', 'Ilya', 'Igor']:
        tasks.append(io_bound_task(name))
        
    try:
        results = await asyncio.gather(*tasks)
    except ValueError:
        print('Error')
    else:
        print(f'COUNT: {sum(results)}')

loop.run_until_complete(main())

RuntimeError: This event loop is already running

group is "Amir"
group is "Zarina"
group is "Misha"
group is "Ilya"
group is "Igor"
Error
count 135
count 181
count 118
count 133
count 111


In [19]:
# Обработка ошибок
async def main():
    tasks = [error_task()]
    
    for name in ['Amir', 'Zarina', 'Misha', 'Ilya', 'Igor']:
        tasks.append(io_bound_task(name))
        
    count = 0
    results = await asyncio.gather(*tasks, return_exceptions=True)
    # TODO Посчитать сумму участников 
            
    print(f'COUNT: {count}')

loop.run_until_complete(main())

group is "Zarina"
group is "Igor"
group is "Misha"
group is "Ilya"
group is "Amir"
count 155
count 149
count 122
count 183
count 120
count 160
count 176
count 152
count 110
count 146
COUNT: 0


# Итераторы

In [41]:
# Утиная типизация
class FibIterator:
    def __init__(self, n):
        self.n = n
        
    def __iter__(self):
        self.prev, self.cur = 0, 1
        self.ind = 0
        return self
    
    def __next__(self):
        if self.ind == self.n:
            raise StopIteration
    
        self.ind += 1
        self.prev, self.cur = self.cur, self.cur + self.prev
        return self.ind, self.prev

for ind, num in FibIterator(20):
    print(f'{ind} -> {num}')

1 -> 1
2 -> 1
3 -> 2
4 -> 3
5 -> 5
6 -> 8
7 -> 13
8 -> 21
9 -> 34
10 -> 55
11 -> 89
12 -> 144
13 -> 233
14 -> 377
15 -> 610
16 -> 987
17 -> 1597
18 -> 2584
19 -> 4181
20 -> 6765


In [21]:
# Реализуем свой цикл for
fib_seq = iter(FibIterator(20))
while True:
    try:
        ind, num = next(fib_seq)
        print(f'{ind} -> {num}')
    except StopIteration:
        print('EXIT FROM LOOP')
        break

1 -> 1
2 -> 1
3 -> 2
4 -> 3
5 -> 5
6 -> 8
7 -> 13
8 -> 21
9 -> 34
10 -> 55
11 -> 89
12 -> 144
13 -> 233
14 -> 377
15 -> 610
16 -> 987
17 -> 1597
18 -> 2584
19 -> 4181
20 -> 6765
EXIT FROM LOOP


# Генераторы

In [42]:
# Бесконечный генератор
def fib_generator():
    prev, cur = 0, 1
    while True:
        yield cur
        prev, cur = cur, cur + prev

fib_seq = fib_generator()
for ind in range(1, 21):
    print(f'{ind} -> {next(fib_seq)}')
    
f'OUT LOOP: -> {next(fib_seq)}'

1 -> 1
2 -> 1
3 -> 2
4 -> 3
5 -> 5
6 -> 8
7 -> 13
8 -> 21
9 -> 34
10 -> 55
11 -> 89
12 -> 144
13 -> 233
14 -> 377
15 -> 610
16 -> 987
17 -> 1597
18 -> 2584
19 -> 4181
20 -> 6765


'OUT LOOP: -> 10946'

In [23]:
# Генератор с остановкой
def fib_generator(n):
    prev, cur = 0, 1
    
    ind = 1
    while ind <= n:
        yield ind, cur
        prev, cur = cur, cur + prev
        ind += 1

for ind, val in fib_generator(20):
    print(f'{ind} -> {val}')

1 -> 1
2 -> 1
3 -> 2
4 -> 3
5 -> 5
6 -> 8
7 -> 13
8 -> 21
9 -> 34
10 -> 55
11 -> 89
12 -> 144
13 -> 233
14 -> 377
15 -> 610
16 -> 987
17 -> 1597
18 -> 2584
19 -> 4181
20 -> 6765


In [24]:
# Генератор реализует интерфейс итератора
fib_seq = fib_generator(20)
while True:
    try:
        ind, num = next(fib_seq)
        print(f'{ind} -> {num}')
    except StopIteration:
        print('EXIT FROM LOOP')
        break

1 -> 1
2 -> 1
3 -> 2
4 -> 3
5 -> 5
6 -> 8
7 -> 13
8 -> 21
9 -> 34
10 -> 55
11 -> 89
12 -> 144
13 -> 233
14 -> 377
15 -> 610
16 -> 987
17 -> 1597
18 -> 2584
19 -> 4181
20 -> 6765
EXIT FROM LOOP
